# Artificial Neural Network on Location dataset

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split

In [2]:
loaded = np.load("./data/bb_data.npz")
X_train = loaded["X_train"]
y_train = loaded["y_train"]
X_test = loaded["X_test"]
y_test = loaded["y_test"]

In [3]:
train_target = torch.tensor(y_train, dtype=torch.float64)
train = torch.tensor(X_train, dtype=torch.float64) 
train_tensor = data_utils.TensorDataset(train, train_target) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size=64, shuffle = True)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        ### FOR XAVIER INITIALIZATION
        self.fc1 = nn.Linear(236, 128) #fc stays for 'fully connected'
        nn.init.xavier_normal_(self.fc1.weight)
        self.drop = nn.Dropout(0.3)
        self.fc4 = nn.Linear(128,30)
        nn.init.xavier_normal_(self.fc4.weight)
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = torch.tanh(self.fc1(x))
        x = self.fc4(self.drop(x))
        return F.softmax(x, dim=1)
    
    
net=Net()
net=net.float()
print(net)

Net(
  (fc1): Linear(in_features=236, out_features=128, bias=True)
  (drop): Dropout(p=0.3, inplace=False)
  (fc4): Linear(in_features=128, out_features=30, bias=True)
)


In [5]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, eps=1e-8, weight_decay=1e-07)
print('inizio training rete')


inizio training rete


In [6]:
for _ in range(50): # 200 full passes over the data
    for data in train_loader:  #is a batch of data
        X, y = data  # X is the batch of features, y is the batch of targets.
        X = X.float()
        y = y.float()
        X=X.view(-1,236)
        net.zero_grad()  # sets gradients to 0 before loss calc. You will do this likely every step.
        output = net(X)# pass in the reshaped batch (recall they are 1*237) view=shape
        loss = loss_function(output, y.long() ) # calc and grab the loss value
        loss.backward() # apply this loss backwards thru the network's parameters
        optimizer.step()  # attempt to optimize weights to account for loss/gradients
    print(loss)  # print loss. We hope loss (a measure of wrong-ness) declines! 

tensor(3.2938, grad_fn=<NllLossBackward>)
tensor(3.0330, grad_fn=<NllLossBackward>)
tensor(3.1915, grad_fn=<NllLossBackward>)
tensor(2.7640, grad_fn=<NllLossBackward>)
tensor(2.7127, grad_fn=<NllLossBackward>)
tensor(2.7687, grad_fn=<NllLossBackward>)
tensor(2.7009, grad_fn=<NllLossBackward>)
tensor(2.7460, grad_fn=<NllLossBackward>)
tensor(2.6360, grad_fn=<NllLossBackward>)
tensor(2.4925, grad_fn=<NllLossBackward>)
tensor(2.6426, grad_fn=<NllLossBackward>)
tensor(2.5696, grad_fn=<NllLossBackward>)
tensor(2.6090, grad_fn=<NllLossBackward>)
tensor(2.5499, grad_fn=<NllLossBackward>)
tensor(2.6105, grad_fn=<NllLossBackward>)
tensor(2.5200, grad_fn=<NllLossBackward>)
tensor(2.5602, grad_fn=<NllLossBackward>)
tensor(2.4825, grad_fn=<NllLossBackward>)
tensor(2.4843, grad_fn=<NllLossBackward>)
tensor(2.5619, grad_fn=<NllLossBackward>)
tensor(2.7269, grad_fn=<NllLossBackward>)
tensor(2.5181, grad_fn=<NllLossBackward>)
tensor(2.5533, grad_fn=<NllLossBackward>)
tensor(2.6282, grad_fn=<NllLossBac

In [7]:
correct = 0
total = 0

with torch.no_grad():
    for data in train_loader:
        X,y = data
        output = net(X.view(-1,236).float()) #sta davvero facendo quello che deve?
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct+=1
            total +=1
print("Accuracy sul train: ", correct/total)

Accuracy sul train:  0.9856792144026186


In [8]:
#TEST SET
test_target = torch.tensor(y_test, dtype=torch.float64)
test = torch.tensor(X_test, dtype=torch.float64) 
test_tensor = data_utils.TensorDataset(test, test_target) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size=16, shuffle = True)


In [9]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        X,y = data
        output = net(X.view(-1,236).float())
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct+=1
            total +=1
print("Accuracy  sul test set: ", round(correct/total,3))

Accuracy  sul test set:  0.842


In [10]:
#stampare il vettore classi - PREDICT -
with torch.no_grad():
  net.eval()
  y_pred=net(test.float())
  #correct = (y_pred.max(dim=1)[1] == y_test)
  #print(torch.mean(correct.item()))
z=[]
for i in y_pred:
    z.append(int(torch.argmax(i)))
print(len(y_pred), len(z))
print('stampo lista predict')
print(len(pd.unique(z)))

612 612
stampo lista predict
30


In [11]:
torch.save(net.state_dict(), "./data/net.pt")
torch.save(optimizer.state_dict(), "./data/optimizer.pt")
torch.save(loss_function.state_dict(), "./data/loss_function.pt")